In [1]:
import itertools
from tqdm import tqdm
import random
import sys
#from numba import jit

In [2]:
%load_ext Cython

In [3]:
#pokerlist是存所有单张扑克的列表
#这里我们2,3,...,J,Q,K,A分别表示为20,30,...,110,120,130,140,黑红梅方表示为1,2,3,4
#相加即的牌的值,这样一个数就表示了牌,并且方便排序
pokerlist=[]
for i in range(2,15):
    for j in range(1,5):
        pokerlist.append(i*10+j)
#pokerlist

In [4]:
#shoulist是存所有手牌组合的列表
shoulist=[]
for i in itertools.combinations(pokerlist,2):
    shoulist.append(i)
#查看下是否正确
# for i in range(20):
#     print(shoulist[i])
len(shoulist)  #1326

1326

In [5]:
#罗列出全部的5牌组合存入列表pokercomb,用于后续计算得分并存入到字典中待查阅
pokercomb=[]
for i in range(len(pokerlist)):
    for j in range(i+1,len(pokerlist)):
        for k in range(j+1,len(pokerlist)):
            for l in range(k+1,len(pokerlist)):
                for m in range(l+1,len(pokerlist)):
                    listtemp=[]
                    listtemp.append(pokerlist[i])
                    listtemp.append(pokerlist[j])
                    listtemp.append(pokerlist[k])
                    listtemp.append(pokerlist[l])
                    listtemp.append(pokerlist[m])
                    pokercomb.append(listtemp)

In [6]:
'''
牌力得分计算(用16进制表示这样只需6位数即可):
因为有9种牌型,为第1比较:所有得分设置为从同花顺到高牌不同牌型最高位为8,7,6,....2,1,0
第2,3,4,5,6次比较的AKQJ10...得分设置为 : EDCBA98765432,当为A12345时,A做为1
里面数值计算前面不能加hex(..),不然存的结果会当str看待,这样排序时是按字符串的排序,结果就不对了
想看得分最后结果,可以在结果前面加hex查看得分是否正确
'''
#pokercomb=[(132, 133, 134, 141, 143)]
pokercombscore=dict()
for i in pokercomb:
    listnum=[i[0]//10,i[1]//10,i[2]//10,i[3]//10,i[4]//10]
    listcol=[i[0]%10,i[1]%10,i[2]%10,i[3]%10,i[4]%10]
    #判断是否同花
    samecolor_temp=[listcol[0]-listcol[1],listcol[1]-listcol[2],listcol[2]-listcol[3],listcol[3]-listcol[4]]
    samecolor=samecolor_temp.count(0)==4
    #判断是否顺子
    flush_temp=[listnum[0]-listnum[1],listnum[1]-listnum[2],listnum[2]-listnum[3],listnum[3]-listnum[4]]
    flush=(flush_temp[0]==-1 and flush_temp[1]==-1 and flush_temp[2]==-1) and (flush_temp[3] in (-1,-9))
    #为同花顺时计算分数
    if samecolor and flush:
        if listnum[4]==14 and listnum[3]==13:
            pokercombscore[tuple(i)]=8*(16**5)+14*(16**4)
        elif listnum[4]==14 and listnum[3]==5:
            pokercombscore[tuple(i)]=8*(16**5)+5*(16**4)
        else:
            pokercombscore[tuple(i)]=8*(16**5)+listnum[4]*(16**4)
    #为4条时计算分数
    elif flush_temp.count(0)==3 and (flush_temp[0]!=0 or flush_temp[3]!=0):
        if flush_temp[0]!=0:
            pokercombscore[tuple(i)]=7*(16**5) + listnum[4]*(16**4) + listnum[0]*(16**3)
        else:
            pokercombscore[tuple(i)]=7*(16**5)+listnum[0]*(16**4)+listnum[4]*(16**3)
    #为葫芦时计算分数
    elif flush_temp.count(0)==3 and (flush_temp[1]!=0 or flush_temp[2]!=0):
        if flush_temp[1]!=0:
            pokercombscore[tuple(i)]=6*(16**5) + listnum[4]*(16**4) + listnum[0]*(16**3)
        else:
            pokercombscore[tuple(i)]=6*(16**5)+listnum[0]*(16**4)+listnum[4]*(16**3)
    #为同花时计算分数
    elif samecolor:
            pokercombscore[tuple(i)]=5*(16**5) + listnum[4]*(16**4) + listnum[3]*(16**3)+listnum[2]*(16**2)+listnum[1]*(16**1)+listnum[0]*(16**0)
    #为顺子时计算分数
    elif flush:
        if listnum[4]==14 and listnum[3]==13:
            pokercombscore[tuple(i)]=4*(16**5)+14*(16**4)
        elif listnum[4]==14 and listnum[3]==5:
            pokercombscore[tuple(i)]=4*(16**5)+5*(16**4)
        else:
            pokercombscore[tuple(i)]=4*(16**5)+listnum[4]*(16**4)
    #为3条时计算分数
    elif flush_temp.count(0)==2 and (flush_temp[0]==0 and flush_temp[1]==0):
        pokercombscore[tuple(i)]=3*(16**5) + listnum[0]*(16**4) + listnum[4]*(16**3)+listnum[3]*(16**2)
    elif flush_temp.count(0)==2 and (flush_temp[1]==0 and flush_temp[2]==0):
        pokercombscore[tuple(i)]=3*(16**5) + listnum[1]*(16**4) + listnum[4]*(16**3)+listnum[0]*(16**2)
    elif flush_temp.count(0)==2 and (flush_temp[2]==0 and flush_temp[3]==0):
        pokercombscore[tuple(i)]=3*(16**5) + listnum[4]*(16**4) + listnum[1]*(16**3)+listnum[0]*(16**2)
    #为两对时计算分数
    elif flush_temp.count(0)==2 and (flush_temp[1]==0 and flush_temp[3]==0):
        pokercombscore[tuple(i)]=2*(16**5) + listnum[4]*(16**4) + listnum[2]*(16**3)+listnum[0]*(16**2)
    elif flush_temp.count(0)==2 and (flush_temp[0]==0 and flush_temp[3]==0):
        pokercombscore[tuple(i)]=2*(16**5) + listnum[4]*(16**4) + listnum[0]*(16**3)+listnum[2]*(16**2)
    elif flush_temp.count(0)==2 and (flush_temp[0]==0 and flush_temp[2]==0):
        pokercombscore[tuple(i)]=2*(16**5) + listnum[3]*(16**4) + listnum[0]*(16**3)+listnum[4]*(16**2)
    #为一对时计算分数
    elif flush_temp.count(0)==1 and flush_temp[0]==0:
        pokercombscore[tuple(i)]=1*(16**5) + listnum[0]*(16**4) + listnum[4]*(16**3)+listnum[3]*(16**2)+listnum[2]*(16**1)
    elif flush_temp.count(0)==1 and flush_temp[1]==0:
        pokercombscore[tuple(i)]=1*(16**5) + listnum[1]*(16**4) + listnum[4]*(16**3)+listnum[3]*(16**2)+listnum[0]*(16**1)
    elif flush_temp.count(0)==1 and flush_temp[2]==0:
        pokercombscore[tuple(i)]=1*(16**5) + listnum[2]*(16**4) + listnum[4]*(16**3)+listnum[1]*(16**2)+listnum[0]*(16**1)
    elif flush_temp.count(0)==1 and flush_temp[3]==0:
        pokercombscore[tuple(i)]=1*(16**5) + listnum[3]*(16**4) + listnum[2]*(16**3)+listnum[1]*(16**2)+listnum[0]*(16**1)
    #为高牌时计算分数
    else:
        pokercombscore[tuple(i)]=0*(16**5) + listnum[4]*(16**4) + listnum[3]*(16**3)+listnum[2]*(16**2)+listnum[1]*(16**1)+listnum[0]*(16**0)
    
#print(pokercombscore)

In [7]:
#测试下看得分是不是对的
for i in range(20):
    key,value = random.choice(list(pokercombscore.items()))
    print(key,hex(value))
#到这里是第一阶段,将所有牌型的得分存入了一个map,可以判断7牌的得分

(23, 24, 74, 83, 123) 0x12c870
(23, 43, 64, 83, 141) 0xe8642
(53, 81, 102, 123, 134) 0xdca85
(21, 22, 71, 91, 143) 0x12e970
(22, 32, 52, 64, 124) 0xc6532
(81, 82, 101, 103, 104) 0x6a8000
(31, 61, 101, 103, 122) 0x1ac630
(43, 54, 74, 92, 104) 0xa9754
(44, 74, 82, 111, 133) 0xdb874
(22, 32, 44, 123, 141) 0xec432
(24, 42, 51, 64, 122) 0xc6542
(51, 54, 83, 101, 131) 0x15da80
(31, 44, 54, 102, 133) 0xda543
(22, 31, 42, 92, 142) 0xe9432
(24, 42, 93, 104, 141) 0xea942
(53, 74, 111, 124, 143) 0xecb75
(21, 24, 52, 103, 123) 0x12ca50
(23, 73, 113, 114, 142) 0x1be720
(34, 42, 82, 113, 122) 0xcb843
(24, 52, 72, 91, 121) 0xc9752


In [8]:
#计算7牌得分
#手牌和公共牌 都不用注意排序,因为传进来后会进行一次排序!!!
def finallscore1(shou,public):
    shou_seven=sorted(shou+public)
    shou_five=max(list(map(lambda x:pokercombscore[x],list(itertools.combinations(shou_seven,5)))))
    return shou_five

In [9]:
hex(finallscore1((141,142),(23,32,42,61,143)))

'0x3e6400'

In [10]:
'''
研究1 :
计算每张牌力,是返回详细盈亏分布情况
例如传入AsKd,返回
0 : (1432,43,53464)
1 : (3453 ,12 ,45647)
...
其中0表示自己是高牌,盈利1432次,平局43次,输掉53464次
分析时再全部换算成概率
'''

def fenbu(shou,n):   #shou为手牌,n为对手数量
    global pokerlist1
    pokerlist1=set(pokerlist) #全部可能单牌转换为集合方便做交并计算,且不污染原值
    iternum=1000000
    dicttemp={0:[0,0,0],1:[0,0,0],2:[0,0,0],3:[0,0,0],4:[0,0,0],5:[0,0,0],6:[0,0,0],7:[0,0,0],8:[0,0,0]}
    temptemp=0
    for j in tqdm(range(iternum)):
        duis=[] #存储所有生成的对手的手牌
        duis_score=[] #存储所有对手的得分
        temppokerlist1=pokerlist1-set(shou)  #对手可能的单牌
        for i in range(n):
            duis.append(tuple(random.sample(pokerlist1,2)))  #随机抽2个作为一个对手手牌
            temppokerlist1=temppokerlist1-set(duis[i])
        public=tuple(random.sample(temppokerlist1,5))  #随机抽5个作为公共牌
        for i in range(n):
            duis_score.append(finallscore1(duis[i],public))   #该对手手牌评分
        shou_score=finallscore1(shou,public)
        shou_paixing=shou_score//(16**5)  #手牌的牌型
        #判断结果,判断手牌牌型
        if shou_score>max(duis_score):  #胜利
            dicttemp[shou_paixing][0]+=1/iternum
        elif shou_score==max(duis_score):
            dicttemp[shou_paixing][1]+=1/iternum
        elif shou_score<max(duis_score):
            dicttemp[shou_paixing][2]+=1/iternum
    return shou,dicttemp
            
# a=fenbu((141,142),1)
# a

In [11]:
#测试速度用
a=fenbu((141,142),1)
a

100%|█████████████████████████████████████████████████████████████████████| 1000000/1000000 [00:37<00:00, 26849.48it/s]


((141, 142),
 {0: [0, 0, 0],
  1: [0.28902599999918516, 0.0017299999999999538, 0.06914200000004848],
  2: [0.3378099999978799, 0.0020389999999999462, 0.056596000000035944],
  3: [0.10787500000008722, 0.00026599999999999996, 0.009532999999999448],
  4: [0.006354000000000519, 0.004409000000000261, 0.0016019999999999646],
  5: [0.017497999999996846, 0.0015209999999999714, 0.000639000000000009],
  6: [0.08402100000006336, 0.00030300000000000086, 0.0011240000000000048],
  7: [0.008362000000000308, 5.199999999999995e-05, 0],
  8: [7.09999999999999e-05, 2.4000000000000007e-05, 3e-06]})

In [20]:
%%cython
pokerlist1=set(pokerlist) #全部可能单牌转换为集合方便做交并计算,且不污染原值
iternum=1000000
dicttemp={0:[0,0,0],1:[0,0,0],2:[0,0,0],3:[0,0,0],4:[0,0,0],5:[0,0,0],6:[0,0,0],7:[0,0,0],8:[0,0,0]}
temptemp=0
for j in tqdm(range(iternum)):
    duis=[] #存储所有生成的对手的手牌
    duis_score=[] #存储所有对手的得分
    temppokerlist1=pokerlist1-set((141,142))  #对手可能的单牌
    for i in range(1):
        duis.append(tuple(random.sample(pokerlist1,2)))  #随机抽2个作为一个对手手牌
        temppokerlist1=temppokerlist1-set(duis[i])
    public=tuple(random.sample(temppokerlist1,5))  #随机抽5个作为公共牌
    for i in range(1):
        duis_score.append(finallscore1(duis[i],public))   #该对手手牌评分
    shou_score=finallscore1((141,142),public)
    shou_paixing=shou_score//(16**5)  #手牌的牌型
    #判断结果,判断手牌牌型
    if shou_score>max(duis_score):  #胜利
        dicttemp[shou_paixing][0]+=1/iternum
    elif shou_score==max(duis_score):
        dicttemp[shou_paixing][1]+=1/iternum
    elif shou_score<max(duis_score):
        dicttemp[shou_paixing][2]+=1/iternum


Error compiling Cython file:
------------------------------------------------------------
...
pokerlist1=set(pokerlist) #全部可能单牌转换为集合方便做交并计算,且不污染原值
              ^
------------------------------------------------------------

C:\Users\great\.ipython\cython\_cython_magic_7c6fc74e4360661c5a62d052ef3a35a4.pyx:1:15: undeclared name not builtin: pokerlist

Error compiling Cython file:
------------------------------------------------------------
...
pokerlist1=set(pokerlist) #全部可能单牌转换为集合方便做交并计算,且不污染原值
iternum=1000000
dicttemp={0:[0,0,0],1:[0,0,0],2:[0,0,0],3:[0,0,0],4:[0,0,0],5:[0,0,0],6:[0,0,0],7:[0,0,0],8:[0,0,0]}
temptemp=0
for j in tqdm(range(iternum)):
        ^
------------------------------------------------------------

C:\Users\great\.ipython\cython\_cython_magic_7c6fc74e4360661c5a62d052ef3a35a4.pyx:5:9: undeclared name not builtin: tqdm

Error compiling Cython file:
------------------------------------------------------------
...
for j in tqdm(range(iternum)):
    duis=[] #存储所有生成的对

In [ ]:
#测试cython看能不能加速
%%cython
cdef long iternum

def fenbu1(shou,n):   #shou为手牌,n为对手数量
    global pokerlist1
    pokerlist1=set(pokerlist) #全部可能单牌转换为集合方便做交并计算,且不污染原值
    iternum=1000000
    dicttemp={0:[0,0,0],1:[0,0,0],2:[0,0,0],3:[0,0,0],4:[0,0,0],5:[0,0,0],6:[0,0,0],7:[0,0,0],8:[0,0,0]}
    temptemp=0
    for j in tqdm(range(iternum)):
        duis=[] #存储所有生成的对手的手牌
        duis_score=[] #存储所有对手的得分
        temppokerlist1=pokerlist1-set(shou)  #对手可能的单牌
        for i in range(n):
            duis.append(tuple(random.sample(pokerlist1,2)))  #随机抽2个作为一个对手手牌
            temppokerlist1=temppokerlist1-set(duis[i])
        public=tuple(random.sample(temppokerlist1,5))  #随机抽5个作为公共牌
        for i in range(n):
            duis_score.append(finallscore1(duis[i],public))   #该对手手牌评分
        shou_score=finallscore1(shou,public)
        shou_paixing=shou_score//(16**5)  #手牌的牌型
        #判断结果,判断手牌牌型
        if shou_score>max(duis_score):  #胜利
            dicttemp[shou_paixing][0]+=1/iternum
        elif shou_score==max(duis_score):
            dicttemp[shou_paixing][1]+=1/iternum
        elif shou_score<max(duis_score):
            dicttemp[shou_paixing][2]+=1/iternum
    return shou,dicttemp
            
a=fenbu1((141,142),1)
a

In [10]:
shoulist_new=[] #1326张牌简化成对子,同花异,杂花异共169种牌型
for i in shoulist:
    if i[0]//10==i[1]//10 and i[0]%10==1 and i[1]%10==2:
            shoulist_new.append(i)
    elif i[0]//10!=i[1]//10 and i[0]%10==1 and (i[1]%10==1 or i[1]%10==2):
            shoulist_new.append(i)
len(shoulist_new),shoulist_new

(169,
 [(21, 22),
  (21, 31),
  (21, 32),
  (21, 41),
  (21, 42),
  (21, 51),
  (21, 52),
  (21, 61),
  (21, 62),
  (21, 71),
  (21, 72),
  (21, 81),
  (21, 82),
  (21, 91),
  (21, 92),
  (21, 101),
  (21, 102),
  (21, 111),
  (21, 112),
  (21, 121),
  (21, 122),
  (21, 131),
  (21, 132),
  (21, 141),
  (21, 142),
  (31, 32),
  (31, 41),
  (31, 42),
  (31, 51),
  (31, 52),
  (31, 61),
  (31, 62),
  (31, 71),
  (31, 72),
  (31, 81),
  (31, 82),
  (31, 91),
  (31, 92),
  (31, 101),
  (31, 102),
  (31, 111),
  (31, 112),
  (31, 121),
  (31, 122),
  (31, 131),
  (31, 132),
  (31, 141),
  (31, 142),
  (41, 42),
  (41, 51),
  (41, 52),
  (41, 61),
  (41, 62),
  (41, 71),
  (41, 72),
  (41, 81),
  (41, 82),
  (41, 91),
  (41, 92),
  (41, 101),
  (41, 102),
  (41, 111),
  (41, 112),
  (41, 121),
  (41, 122),
  (41, 131),
  (41, 132),
  (41, 141),
  (41, 142),
  (51, 52),
  (51, 61),
  (51, 62),
  (51, 71),
  (51, 72),
  (51, 81),
  (51, 82),
  (51, 91),
  (51, 92),
  (51, 101),
  (51, 102),
  

In [ ]:
resultlist=[]
for i in shoulist_new:
    resultlist.append(fenbu(i,1))

 42%|█████████████████████████████▋                                        | 424311/1000000 [00:16<00:22, 25401.36it/s]

In [12]:
import pickle
pickle.dump(resultlist,open('resultlist.pickle','wb'))
temppickle=pickle.load(open('resultlist.pickle','rb'))
temppickle

[((21, 22),
  {0: [0, 0, 0],
   1: [0.1231000000000025, 0.0017000000000000006, 0.22949999999999104],
   2: [0.17129999999999745, 0.01579999999999996, 0.21199999999999297],
   3: [0.11050000000000214, 0.0005, 0.010399999999999993],
   4: [0.005300000000000001, 0.004599999999999999, 0.0022],
   5: [0.01399999999999997, 0.0018000000000000006, 0.0051],
   6: [0.07340000000000108, 0.0022, 0.008600000000000003],
   7: [0.007600000000000007, 0.0001, 0.00030000000000000003],
   8: [0, 0, 0]}),
 ((21, 31),
  {0: [0, 0.009799999999999996, 0.16879999999999773],
   1: [0.07470000000000111, 0.031799999999999884, 0.32019999999998106],
   2: [0.10750000000000205, 0.017999999999999947, 0.09290000000000163],
   3: [0.0256999999999999, 0.0037999999999999974, 0.013599999999999973],
   4: [0.034999999999999976, 0.005800000000000002, 0.0049],
   5: [0.05760000000000062, 0.0021000000000000003, 0.0035999999999999977],
   6: [0.017999999999999947, 0.0022, 0.0020000000000000005],
   7: [0.0009000000000000002, 

In [16]:
import numpy as np
import pandas as pd
column=['高牌胜','高牌平','高牌负'
        ,'一对胜','一对平','一对负'
        ,'两对胜','两对平','两对负'
        ,'三条胜','三条平','三条负'
        ,'顺子胜','顺子平','顺子负'
        ,'同花胜','同花平','同花负'
        ,'葫芦胜','葫芦平','葫芦负'
        ,'四条胜','四条平','四条负'
        ,'同花顺胜','同花顺平','同花顺负']
index=shoulist_new
data=[]
for i in range(len(temppickle)):
    temp=[]
    for j in range(9):
        temp.extend(temppickle[i][1][j])
    data.append(temp)
df=pd.DataFrame(data,columns=column,index=index)
df['胜率']=df['高牌胜']+df['一对胜']+df['两对胜']+df['三条胜']+df['顺子胜']+df['同花胜']+df['葫芦胜']+df['四条胜']+df['同花顺胜']
df['平率']=df['高牌平']+df['一对平']+df['两对平']+df['三条平']+df['顺子平']+df['同花平']+df['葫芦平']+df['四条平']+df['同花顺平']
df['败率']=df['高牌负']+df['一对负']+df['两对负']+df['三条负']+df['顺子负']+df['同花负']+df['葫芦负']+df['四条负']+df['同花顺负']
df['切分胜率']=df['胜率']+df['平率']/2
df['切分败率']=df['败率']+df['平率']/2
df

,高牌胜,高牌平,高牌负,一对胜,一对平,一对负,两对胜,两对平,两对负,三条胜,...,四条平,四条负,同花顺胜,同花顺平,同花顺负,胜率,平率,败率,切分胜率,切分败率
"(21, 22)",0.0000,0.0000,0.0000,0.1231,0.0017,0.2295,0.1713,0.0158,0.2120,0.1105,...,0.0001,0.0003,0.0000,0.0000,0.0,0.5052,0.0267,0.4681,0.51855,0.48145
"(21, 31)",0.0000,0.0098,0.1688,0.0747,0.0318,0.3202,0.1075,0.0180,0.0929,0.0257,...,0.0001,0.0001,0.0009,0.0000,0.0,0.3203,0.0736,0.6061,0.35710,0.64290
"(21, 32)",0.0000,0.0093,0.1810,0.0783,0.0288,0.3392,0.1083,0.0211,0.0977,0.0251,...,0.0000,0.0002,0.0002,0.0000,0.0,0.2828,0.0702,0.6470,0.31790,0.68210
"(21, 41)",0.0000,0.0079,0.1564,0.0841,0.0304,0.3185,0.1088,0.0205,0.0906,0.0257,...,0.0000,0.0001,0.0010,0.0000,0.0,0.3366,0.0724,0.5910,0.37280,0.62720
"(21, 42)",0.0000,0.0085,0.1832,0.0827,0.0310,0.3275,0.1174,0.0202,0.0947,0.0238,...,0.0003,0.0000,0.0002,0.0000,0.0,0.2956,0.0744,0.6300,0.33280,0.66720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(121, 142)",0.0581,0.0032,0.1271,0.2835,0.0063,0.1653,0.1822,0.0092,0.0395,0.0380,...,0.0002,0.0000,0.0002,0.0000,0.0,0.6264,0.0298,0.3438,0.64130,0.35870
"(131, 132)",0.0000,0.0000,0.0000,0.2842,0.0016,0.0836,0.3260,0.0020,0.0670,0.1079,...,0.0000,0.0000,0.0002,0.0001,0.0,0.8250,0.0096,0.1654,0.82980,0.17020
"(131, 141)",0.0572,0.0023,0.1192,0.2785,0.0052,0.1537,0.1844,0.0050,0.0313,0.0375,...,0.0001,0.0000,0.0005,0.0000,0.0,0.6647,0.0202,0.3151,0.67480,0.32520
"(131, 142)",0.0627,0.0027,0.1258,0.2945,0.0055,0.1653,0.1813,0.0063,0.0348,0.0395,...,0.0001,0.0000,0.0002,0.0000,0.0,0.6388,0.0243,0.3369,0.65095,0.34905


In [17]:
df.to_excel('1.xlsx')

In [24]:
temppickle=pickle.load(open('resultlist.pickle','rb'))
temppickle

[((21, 22),
  {0: [0, 0, 0],
   1: [0.1320000000000001, 0.003, 0.23000000000000018],
   2: [0.17900000000000013, 0.015000000000000006, 0.18200000000000013],
   3: [0.11700000000000009, 0, 0.009000000000000001],
   4: [0.002, 0.007, 0.002],
   5: [0.010000000000000002, 0.001, 0.009000000000000001],
   6: [0.07900000000000006, 0.001, 0.012000000000000004],
   7: [0.009000000000000001, 0, 0],
   8: [0, 0.001, 0]}),
 ((21, 23),
  {0: [0, 0, 0],
   1: [0.1290000000000001, 0.002, 0.23200000000000018],
   2: [0.17400000000000013, 0.01800000000000001, 0.22200000000000017],
   3: [0.08900000000000007, 0, 0.012000000000000004],
   4: [0.003, 0.003, 0.003],
   5: [0.013000000000000005, 0.005, 0.005],
   6: [0.06900000000000005, 0.003, 0.005],
   7: [0.011000000000000003, 0, 0.002],
   8: [0, 0, 0]}),
 ((21, 24),
  {0: [0, 0, 0],
   1: [0.1220000000000001, 0.002, 0.23400000000000018],
   2: [0.17300000000000013, 0.015000000000000006, 0.22100000000000017],
   3: [0.10000000000000007, 0, 0.004],
   

[[0,
  0,
  0,
  0.1320000000000001,
  0.003,
  0.23000000000000018,
  0.17900000000000013,
  0.015000000000000006,
  0.18200000000000013,
  0.11700000000000009,
  0,
  0.009000000000000001,
  0.002,
  0.007,
  0.002,
  0.010000000000000002,
  0.001,
  0.009000000000000001,
  0.07900000000000006,
  0.001,
  0.012000000000000004,
  0.009000000000000001,
  0,
  0,
  0,
  0.001,
  0],
 [0,
  0,
  0,
  0.1290000000000001,
  0.002,
  0.23200000000000018,
  0.17400000000000013,
  0.01800000000000001,
  0.22200000000000017,
  0.08900000000000007,
  0,
  0.012000000000000004,
  0.003,
  0.003,
  0.003,
  0.013000000000000005,
  0.005,
  0.005,
  0.06900000000000005,
  0.003,
  0.005,
  0.011000000000000003,
  0,
  0.002,
  0,
  0,
  0],
 [0,
  0,
  0,
  0.1220000000000001,
  0.002,
  0.23400000000000018,
  0.17300000000000013,
  0.015000000000000006,
  0.22100000000000017,
  0.10000000000000007,
  0,
  0.004,
  0.004,
  0.004,
  0.002,
  0.011000000000000003,
  0.002,
  0.002,
  0.074000000000

In [10]:
shoulist

[(21, 22),
 (21, 23),
 (21, 24),
 (21, 31),
 (21, 32),
 (21, 33),
 (21, 34),
 (21, 41),
 (21, 42),
 (21, 43),
 (21, 44),
 (21, 51),
 (21, 52),
 (21, 53),
 (21, 54),
 (21, 61),
 (21, 62),
 (21, 63),
 (21, 64),
 (21, 71),
 (21, 72),
 (21, 73),
 (21, 74),
 (21, 81),
 (21, 82),
 (21, 83),
 (21, 84),
 (21, 91),
 (21, 92),
 (21, 93),
 (21, 94),
 (21, 101),
 (21, 102),
 (21, 103),
 (21, 104),
 (21, 111),
 (21, 112),
 (21, 113),
 (21, 114),
 (21, 121),
 (21, 122),
 (21, 123),
 (21, 124),
 (21, 131),
 (21, 132),
 (21, 133),
 (21, 134),
 (21, 141),
 (21, 142),
 (21, 143),
 (21, 144),
 (22, 23),
 (22, 24),
 (22, 31),
 (22, 32),
 (22, 33),
 (22, 34),
 (22, 41),
 (22, 42),
 (22, 43),
 (22, 44),
 (22, 51),
 (22, 52),
 (22, 53),
 (22, 54),
 (22, 61),
 (22, 62),
 (22, 63),
 (22, 64),
 (22, 71),
 (22, 72),
 (22, 73),
 (22, 74),
 (22, 81),
 (22, 82),
 (22, 83),
 (22, 84),
 (22, 91),
 (22, 92),
 (22, 93),
 (22, 94),
 (22, 101),
 (22, 102),
 (22, 103),
 (22, 104),
 (22, 111),
 (22, 112),
 (22, 113),
 (22,

(169,
 [(21, 22),
  (21, 31),
  (21, 32),
  (21, 41),
  (21, 42),
  (21, 51),
  (21, 52),
  (21, 61),
  (21, 62),
  (21, 71),
  (21, 72),
  (21, 81),
  (21, 82),
  (21, 91),
  (21, 92),
  (21, 101),
  (21, 102),
  (21, 111),
  (21, 112),
  (21, 121),
  (21, 122),
  (21, 131),
  (21, 132),
  (21, 141),
  (21, 142),
  (31, 32),
  (31, 41),
  (31, 42),
  (31, 51),
  (31, 52),
  (31, 61),
  (31, 62),
  (31, 71),
  (31, 72),
  (31, 81),
  (31, 82),
  (31, 91),
  (31, 92),
  (31, 101),
  (31, 102),
  (31, 111),
  (31, 112),
  (31, 121),
  (31, 122),
  (31, 131),
  (31, 132),
  (31, 141),
  (31, 142),
  (41, 42),
  (41, 51),
  (41, 52),
  (41, 61),
  (41, 62),
  (41, 71),
  (41, 72),
  (41, 81),
  (41, 82),
  (41, 91),
  (41, 92),
  (41, 101),
  (41, 102),
  (41, 111),
  (41, 112),
  (41, 121),
  (41, 122),
  (41, 131),
  (41, 132),
  (41, 141),
  (41, 142),
  (51, 52),
  (51, 61),
  (51, 62),
  (51, 71),
  (51, 72),
  (51, 81),
  (51, 82),
  (51, 91),
  (51, 92),
  (51, 101),
  (51, 102),
  

In [ ]:
#传入自己手牌,公共牌,还有所有对手牌,判断输赢
def vsresult1(shou,public,*dui):
    tempvs=1
    myscore=finallscore1(shou,public)
#     print(hex(myscore))
    duiscorelist=[]
    for i in dui:
        #print(type(i))
        duiscorelist.append(finallscore1(tuple(i),public))
#     print(list(map(lambda x:hex(x),duiscorelist)))
    for i in duiscorelist:
        if myscore<i:
            tempvs=-1
            break
        elif myscore==i:
            tempvs=0
    return tempvs
#测试下看是否正确
vsresult1((63,114),(32, 43, 121, 122, 141),*[(61,142),(131,132)])

In [ ]:
#模拟翻牌前制定牌胜率
shou=(141,142)
#剔除掉自己手牌和对手手牌构成的剩余牌库
shengyuchi_first=set(pokerlist)-set(shou)
#设定对手个数
n=5
#然后做统计
vscore=0
tscore=0
fscore=0
dict1=dict()
for i in tqdm(range(1,500001)):
    shengyuchi=shengyuchi_first
    #随机生成n个对手的手牌
    templist=random.sample(shengyuchi,2*n)
    dui=[]
    for j in range(0,2*n,2):
        dui.append((templist[j],templist[j+1]))

    for k in range(len(dui)):
        shengyuchi=set(shengyuchi)-set(dui[k])
        
    public=tuple(sorted(random.sample(shengyuchi,5)))
    temp=vsresult1(shou,public,*dui)
    if temp==1 :
        vscore+=1
    elif temp==-1 :
        fscore+=1
    elif temp==0 :
        tscore+=1
    if i%1000==0:
        dict1[i]=(vscore,tscore,fscore)
        
#绘图外加统计
templist1=[]
templist2=[]
templist3=[]
templist4=[]
for index,item in dict1.items():
    templist1.append(index)
    templist2.append((item[0]+item[1]/2)/(item[0]+item[1]+item[2]))
    templist3.append(item[1]/(item[0]+item[1]+item[2]))
    templist4.append((item[2]+item[1]/2)/(item[0]+item[1]+item[2]))
#templist1
#templist2
# templist3
# templist4

In [ ]:
%matplotlib inline 
#只有加了这句话图表才能在jupyter notebook中加载出来
import matplotlib.pyplot as plt
from matplotlib.pylab import mpl
mpl.rcParams['font.sans-serif'] = ['SimHei'] #解决中文乱码问题
mpl.rcParams['axes.unicode_minus'] = False #解决坐标轴刻度负号乱码
#输出胜率数字
print("{:.2%}".format(templist2[-1]),"{:.2%}".format(templist4[-1]))
#1.创建图表
fig=plt.figure(figsize=(8,6))#创建figure对象
ax=fig.add_subplot(111) #创建axes对象,每个axes对象即一个绘图区域

ax.plot(templist1,templist2,templist1,templist4)
ax.grid(True,linestyle=':',color='r',alpha=0.5)#设置网格线
#4.显示图像
plt.show()

In [ ]:
'''
某张牌的强度=翻牌前的整体胜率,相关的参数有手牌 ,人数
仅研究全推或弃牌情况 : 筹码量/大盲注为多少时 以多大的比例全推才不亏
100%入池胜率低,仅强力牌入池亏大小盲.所以一定有一个平衡点:大于这个平衡点应该紧一点,小于应该松一点.
输入内容到excel中,
'''
存入excel表中,有9张牌,分别为2-9个对手的时候的情况.
列标签分别为'自己手牌值','00','01','02','10','11','12'...其中00表示自己为高牌胜利的概率,01自己为高牌平局概率,02为自己高牌亏的概率
一共31列